<a href="https://colab.research.google.com/github/guilmoreira/mercado_financeiro/blob/main/IBOV_carteira_hipot%C3%A9tica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. INSTALANDO BIBLIOTECAS**



---


```
# Esta etapa irá instalar as bibliotecas necessárias para funcionamento do código
```



In [ ]:
#!pip install -U plotly
#!pip install pandas
#!pip install numpy
!pip install kora -q

     |████████████████████████████████| 60 kB 4.8 MB/s 


##**2. IMPORTANDO BIBLIOTECAS**

---


```
# Importando as bibliotecas
```




In [ ]:
import pandas as pd
import numpy as np
import plotly.express as pl
import time
from time import sleep

#Usando selenium no google colab
from kora.selenium import wd
from datetime import date

#Validando certificado SSL. Não funciona sem esta biblioteca
import ssl


ssl._create_default_https_context = ssl._create_unverified_context

##**3. VARIÁVEIS INICIAIS**

---


```
# Variável de data utilizada devido ao nome do arquivo
```




In [ ]:
#Data de Hoje
today=str(date.today())

##**4. ÍNDICE**

```
# DEFINIR ÍNDICE DE ESTUDO
```
---


In [ ]:
indice_b3='ibov'
indice_b3=indice_b3.upper()
indice_b3

'IBOV'

##**5.FUNÇÃO EXTRAINDO TICKERS POR ÍNDICE**

---



```
# FUNÇÃO QUE EXTRAÍ A CARTEIRA HIPOTÉTICA PARA O ÍNDICE NAQUELA DATA OU PRÓXIMO DIA ÚTIL
# NESTA VERSÃO NÃO FUNCIONA NO SÁBADO OU DOMINGO
# NECESSÁRIO ATUALIZAR O "find_element"
```




5.1 **Referências utilizadas (03/28/2022)**

https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm

https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br


In [ ]:
#Função baixa os arquivos do site

def composicao_indice(indice, delay=10):
#funçao utiliza o selenium para realizar essa tarefa
  
  link=f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice}?language=pt-br'
  
  wd.get(link)

#ATUALIZAR O find_element_by_id para find_element(by=By.ID)?? Pesquisar!
  sleep(5)
  wd.find_element_by_id('segment').send_keys('Setor de Atuação')
  sleep(delay)
  wd.find_element_by_link_text('Download').click()
  sleep(5)
  arquivo=f'/content/{indice.upper()}Dia_{today[8:]}-{today[5:7]}-{today[2:4]}.csv'
  return pd.read_csv(arquivo, sep=';',encoding='ISO-8859-1',skipfooter=2,engine='python',thousands='.',decimal=',',header=1,index_col=False)

##7. **EXTRAÇÃO DOS ARQUIVOS**

---



```
# DOWNLOAD DOS ARQUIVOS PARA OS ÍNDICES.
# GERADA UMA DATABASE (geral) com as carteiras
```



In [ ]:
composicao_indice(indice_b3, delay=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  


,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1484859030,2.188,2.188
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734558205,0.459,0.459
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327741172,0.354,2.284
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115693556,0.682,2.284
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,339237914,0.116,2.284
...,...,...,...,...,...,...,...
86,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,248477689,0.536,6.587
87,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1261568398,0.875,6.587
88,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255217329,0.509,6.587
89,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1128906085,1.303,6.587


In [ ]:
#EXTRAÇÃO SOMENTE UM ÍNDICE
indice_unico='IBOV'
ibov=composicao_indice(str(indice_unico))

MaxRetryError: ignored

In [ ]:
#EXTRAINTO TODOS OS ÍNDICES

tempo_total = time.time()

geral=[]

for i in lista_indices:
  tempo_ini = time.time()
  dados=composicao_indice(str(i))
  tempo_final  = time.time()
  tempo_ind=tempo_final-tempo_ini
  geral.append(dados)
  print(f'O índice {i} foi baixado com sucesso num tempo de {str(tempo_ind)} s. Dimensões: {dados.shape}')

tempo_loop=(time.time())-tempo_total

##8. **TRATANDO OS DADOS**

---


```
# NECESSÁRIO AJUSTE NOS DADOS INDIVIDUALIZADOS
# DEIXEI UM PADRÃO PARA O IBOV
```



In [ ]:
#VERIRICANDO DISCREPÂCIAS
def renomear_setor(setor):
  if setor == 'Cons N Básico' or setor=='Cons N Cíclico': return 'Consumo Não-Cíclico'
  if setor == 'Financ e outros' or setor=='Financeiro e Outros': return 'Financeiro'
  if setor == 'Telecomunicaçã' : return 'Telecomunicação'
  if setor == 'Utilidade Públ' : return 'Utilidde Pública'
  if setor == 'Bens Indls' : return 'Bens e Indústriais'
  if setor == 'Mats Básicos' : return 'Materiais Básicos'
  else: return setor

In [ ]:
#LOOP PARA TRATAR OS DADOS
carteira=[]
tempo_ini = time.time()

for i in range(0,16):
  tempo_ini = time.time()
  tratamento=geral[i]
  #Criando uma coluna de subsetor partindo da barra
  tratamento['SubSetor']=tratamento['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
  #Subscrevendo a coluna de Setor com dados antes da barra
  tratamento['Setor']=tratamento['Setor'].apply(lambda s: s[:s.rfind('/')].strip())
  #Aplicando função
  tratamento['Setor'].apply(renomear_setor)
  carteira.append(tratamento)
  print(f'numero {i}')

tempo_trat=tempo_ini-(time.time())

print(f' O tempo do loop foi de {str(tempo_trat)} s')

##9. **GRÁFICOS**

---



```
# COMO DADOS NÃO ESTÃO AJUSTADOS PARA TODOS OS ÍNDICES
# ALGUNS ÍNDICES PODEM NÃO FUNCIONAR
```



In [ ]:
#lista dos indices
a=0
print('Indice numérico utilizado na figura')
for i in lista_indices:
  print(f'{i.upper()}={a}')
  a=a+1

In [ ]:
#UTILIZAR O ÍNDICE DESEJADO
base=carteira[10]

In [ ]:
base.sort()

In [ ]:
#GRÁFICO DE PIZZA ITERATIVO TENDO QUE AJUSTAR CONFORME DADOS

fig1=pl.sunburst(data_frame=base,path=['Setor','SubSetor','Código'], values='Part. (%)', height=1000)
fig1.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')

In [ ]:
#TREEMAP

fig2=pl.treemap(data_frame=base,path=['Setor','SubSetor','Código'], values='Part. (%)', height=1000)
fig2.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:.2f}%')